In [114]:
#import libraries

import requests
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [115]:
!pip install beautifulsoup4

distributed 1.21.8 requires msgpack, which is not installed.


In [116]:
!pip install lxml

distributed 1.21.8 requires msgpack, which is not installed.


In [117]:
!pip install html5lib

distributed 1.21.8 requires msgpack, which is not installed.


In [118]:
!pip install requests

distributed 1.21.8 requires msgpack, which is not installed.


In [450]:
from bs4 import BeautifulSoup

In [451]:
#Assign website link to a variable
wikipedia_link='https://www.zipcodestogo.com/New%20Jersey/'

In [452]:
#Use the function get from the requests library to download the Wikipedia page using the wikipedia_link as an argument. Assign the object to the variable postal_code_page.

#hint: requests.get()
source = requests.get(wikipedia_link).text
soup= BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [453]:
table = soup.table
#print(table)
postheader = table.find_all('tr')
#print(postheader)
 

In [454]:
! pip install tabulate

distributed 1.21.8 requires msgpack, which is not installed.


Get the data into a dataframe "DATA"

In [455]:
from tabulate import tabulate
table = soup.find_all('table')[1] 
df_list = pd.read_html(str(table))
#df_list[0]
type(df_list)
#print( tabulate(df_list[0], headers='keys', tablefmt='psql') )

data = pd.DataFrame(df_list[0])
type(data)
#data # 290*3

new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

type(data) #289*3
len(data)
data.head()

,Zip Codes for the State of New Jersey,nan,nan,nan
1,Zip Code,City,County,Zip Code Map
2,07001,Avenel,Middlesex,View Map
3,07002,Bayonne,Hudson,View Map
4,07003,Bloomfield,Essex,View Map
5,07004,Fairfield,Essex,View Map


Clean up data where data is NOT ASSIGNED

In [456]:
data = data.reset_index(drop=True)
len(data)
data.head()

,Zip Codes for the State of New Jersey,nan,nan,nan
0,Zip Code,City,County,Zip Code Map
1,07001,Avenel,Middlesex,View Map
2,07002,Bayonne,Hudson,View Map
3,07003,Bloomfield,Essex,View Map
4,07004,Fairfield,Essex,View Map


In [457]:
data.columns = data.iloc[0]
#data = data.reindex(data.index.drop(0)).reset_index(drop=True)
#data.columns.name = None
data.head()


,Zip Code,City,County,Zip Code Map
0,Zip Code,City,County,Zip Code Map
1,07001,Avenel,Middlesex,View Map
2,07002,Bayonne,Hudson,View Map
3,07003,Bloomfield,Essex,View Map
4,07004,Fairfield,Essex,View Map


In [458]:
data_1 = data.drop('Zip Code', axis=1)



In [459]:
data_2 = data_1.drop('Zip Code Map', axis=1)

In [ ]:
data_2.dropna()

In [460]:
data_2.head()
len(data_2)
data_2.head()

,City,County
0,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex


In [461]:
data_2 = data_2.iloc[1:]
data_2.head()

,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex
5,Boonton,Morris


In [462]:
data_2 = data_2.drop_duplicates(subset=['City','County'])
data_2.head()
len(data_2)

584

In [463]:
data_2 = data_2[data_2.City != 'Mc Afee']

In [464]:
len(data_2)

583

In [475]:
data_2 = data_2[data_2.County != 'Sussex' ]

In [477]:
data_2 = data_2[data_2.City != 'Short Hills']

In [478]:
data_2 = data_2.reset_index(drop=True)

In [485]:
print(len(data_2))
data_2.head()

557


,City,County
0,Avenel,Middlesex
1,Bayonne,Hudson
2,Bloomfield,Essex
3,Fairfield,Essex
4,Boonton,Morris


In [486]:
len(data_2)

557

In [492]:
data_2.head()

,City,County
0,Avenel,Middlesex
1,Bayonne,Hudson
2,Bloomfield,Essex
3,Fairfield,Essex
4,Boonton,Morris


In [493]:
column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

In [494]:
for index, row in data_2.iterrows():
    try:
        address_1 = row['City'] 
        address = address_1+","+row['County']+","+"USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['District'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
        pass
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address = row['County']+","+"USA"
        print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        #print(row['District'],address, latitude, longitude)
        n_hood
        pass


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Problem with data or cannot Geocode.
Somerset,USA


Concatenate the latitude and longitude values to the main dataframe to complete the results

In [496]:
len(n_hood)

557

In [497]:
df.drop(df.index, inplace=True)

In [498]:
len(df)

0

In [499]:
df = pd.concat([data_2, n_hood[['Latitude', 'Longitude']]], axis=1)
df.shape

(557, 4)

In [500]:
df.head()

,City,County,Latitude,Longitude
0,Avenel,Middlesex,40.580382,-74.285147
1,Bayonne,Hudson,40.668714,-74.114309
2,Bloomfield,Essex,40.806767,-74.185423
3,Fairfield,Essex,40.883710,-74.305982
4,Boonton,Morris,40.902599,-74.407097


In [501]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['County'].unique()),
        df.shape[0]
    )
)

The dataframe has 20 boroughs and 557 neighborhoods.


In [502]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'New Jersey,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

EXPLORE New Jersey

In [503]:
# create map of New York using latitude and longitude values
map_nj = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['County'], df['City']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nj) 
    
map_nj


n_hood_1 = df
n_hood_1.head()
n_hood_2 = n_hood_1.drop('Postcode', axis=1)
len(n_hood_2)
n_hood_2.head()

In [505]:
essex_data = df[df['County'] == 'Essex'].reset_index(drop=True)
len(essex_data)
essex_data

,City,County,Latitude,Longitude
0,Bloomfield,Essex,40.806767,-74.185423
1,Fairfield,Essex,40.883710,-74.305982
2,Caldwell,Essex,40.839822,-74.276537
3,Cedar Grove,Essex,40.851766,-74.229035
4,East Orange,Essex,40.767323,-74.204868
5,Essex Fells,Essex,40.824544,-74.284593
6,Glen Ridge,Essex,40.805378,-74.203757
7,Livingston,Essex,40.795933,-74.314871
8,Maplewood,Essex,40.731212,-74.273481
9,Millburn,Essex,40.724444,-74.306907


In [506]:
address = 'Essex, New Jersey'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [507]:
# create map of Manhattan using latitude and longitude values
map_essex = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(essex_data['Latitude'], essex_data['Longitude'], essex_data['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_essex)  
    
map_essex

In [508]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret


In [509]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'New Jersey,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


name = 'Essex' # neighborhood name
print(name)

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               latitude, 
                                                               longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


GeocoderQuotaExceeded: HTTP Error 429: Too Many Requests

In [352]:
essex_data.loc[16, 'City']

'Newark'

In [510]:
neighborhood_latitude = essex_data.loc[16, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = essex_data.loc[16, 'Longitude'] # neighborhood longitude value

neighborhood_name = essex_data.loc[16, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Newark are 40.735657, -74.1723667.


In [511]:
# type your answer here
# type your answer here
CLIENT_ID = 'EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW'
CLIENT_SECRET = '3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1'
VERSION = '20180922'
radius = 1000
LIMIT = 500

#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW&client_secret=3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1&v=20180922&ll=40.735657,-74.1723667&radius=1000&limit=500'

In [512]:
results = requests.get(url).json()
#results

In [513]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [514]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues


,name,categories,lat,lng
0,Hobby's Delicatessen,Deli / Bodega,40.735048,-74.174395
1,Fire Lounge,Lounge,40.733834,-74.171394
2,Prudential Center,Hockey Arena,40.733608,-74.171155
3,Blaze Pizza,Pizza Place,40.738387,-74.170809
4,Halsey Deli,Deli / Bodega,40.737610,-74.172935
5,Dinosaur Bar-B-Que,BBQ Joint,40.734755,-74.170207
6,Barnabas Health Hockey House,Skating Rink,40.732685,-74.171708
7,Eleven80,Residential Building (Apartment / Condo),40.737324,-74.170117
8,Dario's Restaurant,Mexican Restaurant,40.737399,-74.172631
9,Cafe Airlie,Sandwich Place,40.736749,-74.169865


In [515]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [516]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #print(nearby_venues)
    return(nearby_venues)

In [517]:
essex_venues = getNearbyVenues(names=essex_data['City'],
                                   latitudes=essex_data['Latitude'],
                                   longitudes=essex_data['Longitude']
                                  )


Bloomfield
Fairfield
Caldwell
Cedar Grove
East Orange
Essex Fells
Glen Ridge
Livingston
Maplewood
Millburn
Montclair
Verona
Orange
West Orange
Roseland
South Orange
Newark
Belleville
Nutley
Irvington


In [518]:
essex_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belleville,22,22,22,22,22,22
Bloomfield,2,2,2,2,2,2
Caldwell,33,33,33,33,33,33
Cedar Grove,24,24,24,24,24,24
East Orange,13,13,13,13,13,13
Essex Fells,1,1,1,1,1,1
Fairfield,1,1,1,1,1,1
Glen Ridge,16,16,16,16,16,16
Irvington,6,6,6,6,6,6


In [519]:
print('There are {} uniques categories.'.format(len(essex_venues['Venue Category'].unique())))

There are 131 uniques categories.


In [520]:
len(essex_data)

20

In [521]:
# one hot encoding
essex_onehot = pd.get_dummies(essex_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
essex_onehot['Neighborhood'] = essex_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [essex_onehot.columns[-1]] + list(essex_onehot.columns[:-1])
essex_onehot = essex_onehot[fixed_columns]

len(essex_onehot)

408

In [522]:
essex_onehot.shape

(408, 132)

In [523]:
essex_grouped = essex_onehot.groupby('Neighborhood').mean().reset_index()
len(essex_grouped)

20

In [524]:
essex_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Garden,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Butcher,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hockey Arena,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kids Store,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Belleville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.090909,0.000000,0.000000,0.000,0.0,0.045455,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.0,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000
1,Bloomfield,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [525]:
num_top_venues = 5

for hood in essex_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = essex_grouped[essex_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Belleville----
                venue  freq
0  Italian Restaurant  0.09
1       Grocery Store  0.09
2          Bagel Shop  0.09
3      Cosmetics Shop  0.09
4   Convenience Store  0.09


----Bloomfield----
                 venue  freq
0                  Gym   0.5
1                Beach   0.5
2  American Restaurant   0.0
3          Pizza Place   0.0
4           Non-Profit   0.0


----Caldwell----
                venue  freq
0         Pizza Place  0.12
1  Italian Restaurant  0.12
2      Cosmetics Shop  0.03
3       Grocery Store  0.03
4         Coffee Shop  0.03


----Cedar Grove----
                venue  freq
0        Liquor Store  0.12
1  Italian Restaurant  0.08
2         Pizza Place  0.08
3                Bank  0.08
4    Sushi Restaurant  0.04


----East Orange----
                 venue  freq
0  American Restaurant  0.15
1         Liquor Store  0.15
2   Chinese Restaurant  0.15
3           Campground  0.08
4        Moving Target  0.08


----Essex Fells----
                 venue 

In [526]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [528]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = essex_grouped['Neighborhood']

for ind in np.arange(essex_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(essex_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
len(neighborhoods_venues_sorted)

20

In [529]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belleville,Convenience Store,Cosmetics Shop,Grocery Store,Italian Restaurant,Bagel Shop,Donut Shop,Salon / Barbershop,Smoke Shop,Supplement Shop,Latin American Restaurant
1,Bloomfield,Gym,Beach,Yoga Studio,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Field,Frozen Yogurt Shop
2,Caldwell,Pizza Place,Italian Restaurant,Pub,Movie Theater,Seafood Restaurant,School,Mexican Restaurant,Butcher,Farmers Market,Chinese Restaurant
3,Cedar Grove,Liquor Store,Italian Restaurant,Bank,Pizza Place,Jewelry Store,Deli / Bodega,Nail Salon,Salon / Barbershop,Sandwich Place,Food & Drink Shop
4,East Orange,American Restaurant,Liquor Store,Chinese Restaurant,Campground,Moving Target,Fish & Chips Shop,Deli / Bodega,Sandwich Place,Seafood Restaurant,Recording Studio
5,Essex Fells,Lake,Yoga Studio,Field,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Farmers Market
6,Fairfield,Historic Site,Yoga Studio,Frozen Yogurt Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field
7,Glen Ridge,Gastropub,Bakery,Baseball Field,Convenience Store,Burger Joint,Flower Shop,Sandwich Place,Field,Shopping Mall,Mobile Phone Shop
8,Irvington,Supermarket,Chinese Restaurant,Asian Restaurant,Liquor Store,Moving Target,Ethiopian Restaurant,Event Space,Farmers Market,Field,Frozen Yogurt Shop
9,Livingston,Ice Cream Shop,Italian Restaurant,Pizza Place,Health & Beauty Service,Chinese Restaurant,Mediterranean Restaurant,Shopping Mall,Sandwich Place,Salon / Barbershop,Coffee Shop


In [530]:
essex_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Garden,Bistro,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Stop,Butcher,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hockey Arena,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kids Store,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Belleville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.090909,0.000000,0.000000,0.000,0.0,0.045455,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.045455,0.0,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000
1,Bloomfield,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [531]:
# set number of clusters
kclusters = 5

essex_grouped_clustering = essex_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(essex_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 1, 2, 0, 0, 0], dtype=int32)

In [532]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belleville,Convenience Store,Cosmetics Shop,Grocery Store,Italian Restaurant,Bagel Shop,Donut Shop,Salon / Barbershop,Smoke Shop,Supplement Shop,Latin American Restaurant
1,Bloomfield,Gym,Beach,Yoga Studio,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Field,Frozen Yogurt Shop
2,Caldwell,Pizza Place,Italian Restaurant,Pub,Movie Theater,Seafood Restaurant,School,Mexican Restaurant,Butcher,Farmers Market,Chinese Restaurant
3,Cedar Grove,Liquor Store,Italian Restaurant,Bank,Pizza Place,Jewelry Store,Deli / Bodega,Nail Salon,Salon / Barbershop,Sandwich Place,Food & Drink Shop
4,East Orange,American Restaurant,Liquor Store,Chinese Restaurant,Campground,Moving Target,Fish & Chips Shop,Deli / Bodega,Sandwich Place,Seafood Restaurant,Recording Studio
5,Essex Fells,Lake,Yoga Studio,Field,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Farmers Market
6,Fairfield,Historic Site,Yoga Studio,Frozen Yogurt Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field
7,Glen Ridge,Gastropub,Bakery,Baseball Field,Convenience Store,Burger Joint,Flower Shop,Sandwich Place,Field,Shopping Mall,Mobile Phone Shop
8,Irvington,Supermarket,Chinese Restaurant,Asian Restaurant,Liquor Store,Moving Target,Ethiopian Restaurant,Event Space,Farmers Market,Field,Frozen Yogurt Shop
9,Livingston,Ice Cream Shop,Italian Restaurant,Pizza Place,Health & Beauty Service,Chinese Restaurant,Mediterranean Restaurant,Shopping Mall,Sandwich Place,Salon / Barbershop,Coffee Shop


In [534]:
essex_merged = essex_data

# add clustering labels
essex_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
essex_merged = essex_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City')

essex_merged.head() # check the last columns!

,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomfield,Essex,40.806767,-74.185423,0,Gym,Beach,Yoga Studio,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Field,Frozen Yogurt Shop
1,Fairfield,Essex,40.883710,-74.305982,3,Historic Site,Yoga Studio,Frozen Yogurt Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field
2,Caldwell,Essex,40.839822,-74.276537,0,Pizza Place,Italian Restaurant,Pub,Movie Theater,Seafood Restaurant,School,Mexican Restaurant,Butcher,Farmers Market,Chinese Restaurant
3,Cedar Grove,Essex,40.851766,-74.229035,0,Liquor Store,Italian Restaurant,Bank,Pizza Place,Jewelry Store,Deli / Bodega,Nail Salon,Salon / Barbershop,Sandwich Place,Food & Drink Shop
4,East Orange,Essex,40.767323,-74.204868,0,American Restaurant,Liquor Store,Chinese Restaurant,Campground,Moving Target,Fish & Chips Shop,Deli / Bodega,Sandwich Place,Seafood Restaurant,Recording Studio


In [537]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(essex_merged['Latitude'], essex_merged['Longitude'], essex_merged['City'], essex_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

 Cluster 1 : Seems to be the most happening place with lots of restaurants, bus facilities, supermarkets etc.,

In [543]:
essex_merged.loc[essex_merged['Cluster Labels'] == 0, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomfield,Gym,Beach,Yoga Studio,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Field,Frozen Yogurt Shop
2,Caldwell,Pizza Place,Italian Restaurant,Pub,Movie Theater,Seafood Restaurant,School,Mexican Restaurant,Butcher,Farmers Market,Chinese Restaurant
3,Cedar Grove,Liquor Store,Italian Restaurant,Bank,Pizza Place,Jewelry Store,Deli / Bodega,Nail Salon,Salon / Barbershop,Sandwich Place,Food & Drink Shop
4,East Orange,American Restaurant,Liquor Store,Chinese Restaurant,Campground,Moving Target,Fish & Chips Shop,Deli / Bodega,Sandwich Place,Seafood Restaurant,Recording Studio
7,Livingston,Ice Cream Shop,Italian Restaurant,Pizza Place,Health & Beauty Service,Chinese Restaurant,Mediterranean Restaurant,Shopping Mall,Sandwich Place,Salon / Barbershop,Coffee Shop
8,Maplewood,Gift Shop,Pizza Place,Coffee Shop,Ice Cream Shop,Toy / Game Store,Mexican Restaurant,Deli / Bodega,Restaurant,Chinese Restaurant,Pub
9,Millburn,Italian Restaurant,French Restaurant,Pizza Place,Coffee Shop,New American Restaurant,Deli / Bodega,Ice Cream Shop,Gastropub,Plaza,Playground
10,Montclair,Park,Chinese Restaurant,Event Space,Athletics & Sports,Train Station,Bookstore,Theater,Café,Bank,French Restaurant
11,Verona,Deli / Bodega,Pizza Place,Convenience Store,Bus Stop,Gift Shop,Pharmacy,Smoke Shop,Gym / Fitness Center,American Restaurant,Italian Restaurant
12,Orange,Gym,Sandwich Place,Bakery,Shoe Store,Southern / Soul Food Restaurant,Hobby Shop,Latin American Restaurant,Donut Shop,Deli / Bodega,Construction & Landscaping


Cluster 2: Must be a place with a lake to relax and have some food

In [544]:
essex_merged.loc[essex_merged['Cluster Labels'] == 1, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Essex Fells,Lake,Yoga Studio,Field,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Flower Shop,Fish & Chips Shop,Farmers Market


Clutser 3: Must be a place for pub and baseball enthusiasts to hang around

In [545]:
essex_merged.loc[essex_merged['Cluster Labels'] == 2, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Glen Ridge,Gastropub,Bakery,Baseball Field,Convenience Store,Burger Joint,Flower Shop,Sandwich Place,Field,Shopping Mall,Mobile Phone Shop


Cluster 4: Good for a day's outing to see the old world charm

In [546]:
essex_merged.loc[essex_merged['Cluster Labels'] == 3, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Fairfield,Historic Site,Yoga Studio,Frozen Yogurt Shop,Diner,Discount Store,Donut Shop,Ethiopian Restaurant,Event Space,Farmers Market,Field


Cluster 5: Place for shopping and eatout

In [547]:
essex_merged.loc[essex_merged['Cluster Labels'] == 4, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Irvington,Supermarket,Chinese Restaurant,Asian Restaurant,Liquor Store,Moving Target,Ethiopian Restaurant,Event Space,Farmers Market,Field,Frozen Yogurt Shop
